In [ ]:
import MySQLdb as mdb
import csv
import pandas as pd


#configure pandas to show all rows
pd.set_option('display.max_rows', 999)


In [ ]:
#load db
con = mdb.connect('localhost', 'hlugo', 'pass12345', 'destiny');

#create connection
with con:
    cur = con.cursor()

# Search for a players clan

In [ ]:
#Find clan of specific player

#ask for Gamer ID
gamerSearch = raw_input("Enter the Gamer ID of the player you want to find: ")
cur.execute("select gamerID, Player.clanName, Founder, Size, Motto from Player, Clan WHERE Player.clanName = Clan.clanName AND gamerID = %s", [gamerSearch]);
clanRows = cur.fetchall()



#Clan searches
dfClan = pd.DataFrame( [[ij for ij in i] for i in clanRows] )
dfClan.rename(columns={0: 'Gamer ID', 1: 'Clan Name', 2: 'Clan Founder', 3: 'Clan Size', 4: 'Clan Motto'}, inplace=True)
#df = df.sort(['Size'], ascending=[0]);
dfClan.head(99)

In [ ]:
#User input for min/max on light and level
minLevel = raw_input("What is the minimum level desired? :")
maxLevel = raw_input("What is the maximum level desired? :")
minLight = raw_input("What is the minimum light desired? :")
maxLight = raw_input("What is the maximum light desired? :")
findClass = raw_input("What class is desired? :")
#Best equipped characters
cur.execute("SELECT * FROM charView WHERE (level BETWEEN %s AND %s) AND (light BETWEEN %s AND %s) AND class = %s", [minLevel, maxLevel, minLight, maxLight, findClass]);
#cur.execute("select * from pCharacter NATURAL JOIN Player");
rows = cur.fetchall()

#Finding players to play with in specific level and light range
dfr = pd.DataFrame( [[ij for ij in i] for i in rows] )
dfr.rename(columns={0: 'Gamer ID', 1: 'Clan Name', 2: 'Level', 3: 'Race', 4: 'Class', 5: 'Light'}, inplace=True)
dfr = dfr.sort_values(by=['Light'], ascending=[0]);
dfr.head(99)


In [ ]:
#find build of characters from a specific gamer
gamerBuild = raw_input("Enter the Gamer ID of the player whose build you wish to see: ")
cur.execute("select class, pName, sName, hName, helmName, gauntletName, chestName, legName from charBuild WHERE gamerID=%s", [gamerBuild]);
gamerBuilds = cur.fetchall()

#Player build
pBuild = pd.DataFrame( [[ij for ij in i] for i in gamerBuilds] )
print 'Build for Player ' + gamerBuild
pBuild.rename(columns={0: 'Class', 1: 'Primary', 2: 'Special', 3: 'Heavy', 4: 'Helmet', 5: 'Gauntlet', 6: 'Chest Armor', 7: 'Leg Armor'}, inplace=True)
pBuild.head(3)

In [ ]:
cur.execute("CREATE OR REPLACE VIEW pwStats(gamerID, characterNum, class, pName, pAttack, sName, sAttack, hName, hAttack, helmet, helmDef, gauntlet, gauntDef, chestArmor, chestDef, legArmor, legDef) AS select Player.gamerID, charBuild.characterNum, charBuild.class, charBuild.pName, PrimaryWeapons.attack, charBuild.sName, SpecialWeapons.attack, charBuild.hName, HeavyWeapons.attack, charBuild.helmName, Helmet.def, charBuild.gauntletName, Gauntlets.def, charBuild.chestName, ChestArmor.def, charBuild.legName, LegArmor.def from charBuild, PrimaryWeapons, SpecialWeapons, HeavyWeapons, Helmet, Gauntlets, ChestArmor, LegArmor, Player where charBuild.gamerID=Player.gamerID and charBuild.pName=PrimaryWeapons.pName and charBuild.sName=SpecialWeapons.sName and charBuild.hName=HeavyWeapons.hName and charBuild.helmName=Helmet.helmName and charBuild.gauntletName=Gauntlets.gauntletName and charBuild.chestName=ChestArmor.chestName and charBuild.legName=LegArmor.legName;")

In [ ]:
#find lowest equipped weapon
gamerStat = raw_input("Enter Gamer ID: ")
cur.execute("SELECT characterNum, class, pName, pAttack, sName, sAttack, hName, hAttack from pwStats where gamerID=%s", [gamerStat])
pWeaponStats = cur.fetchall()



gs = pd.DataFrame( [[ij for ij in i] for i in pWeaponStats] )
gs.rename(columns={0: 'Character#', 1: 'Class', 2: 'Primary', 3:'Primary Attack', 4:'Special', 5:'Special Attack', 6:'Heavy', 7:'Heavy Attack'}, inplace=True)
gs.head(10)

In [ ]:
#find lowest equipped Armor
gamerStatA = raw_input("Enter Gamer ID: ")
cur.execute("SELECT characterNum, class, helmet, helmDef, gauntlet, gauntDef, chestArmor, chestDef, legArmor, legDef from pwStats where gamerID=%s", [gamerStatA])
pArmorStats = cur.fetchall()



gsa = pd.DataFrame( [[ij for ij in i] for i in pArmorStats] )
gsa.rename(columns={0: 'Character#', 1: 'Class', 2: 'Helmet', 3:'Helmet Defense', 4:'Gauntlet', 5:'Gauntlet Defense', 6:'Chest Armor', 7:'Chest Defense', 8:'Leg Armor', 9: 'Leg Defense'}, inplace=True)
gsa.head(10)

In [ ]:
cur.execute("SELECT pName as Weapon from pwStats where gamerID='wildheck' UNION SELECT sName as Weapon from pwStats where gamerID='wildheck';")

In [ ]:
SELECT characterNum, pName as Weapon, pAttack as Attack from pwStats where gamerID='wildheck' UNION All SELECT characterNum, sName as Weapon, sAttack as Attack from pwStats where gamerID='wildheck' UNION ALL SELECT characterNum, hName as Weapon, hAttack as Attack from pwStats where gamerID='wildheck' order by characterNum;

In [ ]:
#find lowest equiped Weapon
gamerIDs = raw_input("Enter Gamer ID: ")
#cNum = raw_input("Ender the Gamer's Character#: ")
cur.execute("CREATE OR REPLACE VIEW spStats AS SELECT characterNum, class, pName as Weapon, pAttack as Attack from pwStats where gamerID=%s UNION All SELECT characterNum, class, sName as Weapon, sAttack as Attack from pwStats where gamerID=%s UNION ALL SELECT characterNum, class, hName as Weapon, hAttack as Attack from pwStats where gamerID=%s", [gamerIDs, gamerIDs, gamerIDs])
cur.execute("CREATE OR REPLACE VIEW saStats AS SELECT characterNum, class, helmet as Armor, helmDef as Defense from pwStats where gamerID=%s UNION All SELECT characterNum, class, gauntlet as Armor, gauntDef as Defense from pwStats where gamerID=%s UNION ALL SELECT characterNum, class, chestArmor as Armor, chestDef as Defense from pwStats where gamerID=%s UNION ALL SELECT characterNum, class, legArmor as Armor, legDef as Defense from pwStats where gamerID=%s", [gamerIDs, gamerIDs, gamerIDs, gamerIDs])

cur.execute("SELECT spStats.characterNum, spStats.class, spStats.Weapon, min(spStats.Attack), saStats.Armor, min(saStats.Defense) FROM spStats, saStats where spStats.characterNum=1 and saStats.characterNum=1 and Attack=(SELECT min(Attack) from spStats where characterNum=1) and Defense=(SELECT min(Defense) from saStats where characterNum=1)")
gamerStats1 = cur.fetchall()
cur.execute("SELECT spStats.characterNum, spStats.class, spStats.Weapon, min(spStats.Attack), saStats.Armor, min(saStats.Defense) FROM spStats, saStats where spStats.characterNum=2 and saStats.characterNum=2 and Attack=(SELECT min(Attack) from spStats where characterNum=2) and Defense=(SELECT min(Defense) from saStats where characterNum=2)")
gamerStats2 = cur.fetchall()
cur.execute("SELECT spStats.characterNum, spStats.class, spStats.Weapon, min(spStats.Attack), saStats.Armor, min(saStats.Defense) FROM spStats, saStats where spStats.characterNum=3 and saStats.characterNum=3 and Attack=(SELECT min(Attack) from spStats where characterNum=3) and Defense=(SELECT min(Defense) from saStats where characterNum=3)")
gamerStats3 = cur.fetchall()

tv1 = pd.DataFrame( [[ij for ij in i] for i in gamerStats1] )
tv2 = pd.DataFrame( [[ij for ij in i] for i in gamerStats2] )
tv3 = pd.DataFrame( [[ij for ij in i] for i in gamerStats3] )
frames = [tv1, tv2, tv3]
tv = pd.concat(frames)
tv.rename(columns={0: 'Character#', 1: 'Class', 2: 'Weakest Weapon', 3: 'Attack', 4: 'Weakest Armor', 5: 'Defense'}, inplace=True)
tv.head(10)

In [ ]:
#find lowest equipped weapon
gamerStat2 = raw_input("Enter Gamer ID: ")
cur.execute("SELECT characterNum, class, LEAST(pAttack, sAttack, hAttack) from pwStats where gamerID=%s", [gamerStat2])
pWeaponStats2 = cur.fetchall()



gs2 = pd.DataFrame( [[ij for ij in i] for i in pWeaponStats2] )
gs2.rename(columns={0: 'Character#', 1: 'Class', 2: 'Least Effective Weapon'}, inplace=True)
gs2.head(10)

In [ ]:
#find primary
pRarity = raw_input("What Primary Weapon rarity are you searching for: ")
cur.execute("SELECT pName, subtype, attack FROM PrimaryWeapons WHERE rarity=%s", [pRarity])
pWeapons = cur.fetchall()



pW = pd.DataFrame( [[ij for ij in i] for i in pWeapons] )
pW.rename(columns={0: 'Primary Weapon Name', 1: 'Subtype', 2: 'Attack Value'}, inplace=True)
pW.head(200)

In [ ]:
#find special
sRarity = raw_input("What Special Weapon rarity are you searching for: ")
cur.execute("SELECT sName, subtype, attack FROM SpecialWeapons WHERE rarity=%s", [sRarity])
sWeapons = cur.fetchall()



sW = pd.DataFrame( [[ij for ij in i] for i in sWeapons] )
sW.rename(columns={0: 'Speicla Weapon Name', 1: 'Subtype', 2: 'Attack Value'}, inplace=True)
sW.head(200)

In [ ]:
#find primary

hRarity = raw_input("What Heavy Weapon rarity are you searching for: ")
cur.execute("SELECT hName, subtype, attack FROM HeavyWeapons WHERE rarity=%s", [hRarity])
hWeapons = cur.fetchall()



hW = pd.DataFrame( [[ij for ij in i] for i in hWeapons] )
hW.rename(columns={0: 'Heavy Weapon Name', 1: 'Subtype', 2: 'Attack Value'}, inplace=True)
hW.head(200)

In [ ]:
# find players quests
qGamer = raw_input("What Gamer ID are you searching for: ")
cur.execute("SELECT characterNum, race, class, level, title FROM gamerQuests WHERE gamerID=%s", [qGamer])
quests = cur.fetchall()


qg = pd.DataFrame( [[ij for ij in i] for i in quests] )
qg.rename(columns={0: 'Character#',1: 'Race', 2: 'Class', 3: 'Level', 4: 'Quests Title' }, inplace=True)
qg.head(36)

In [ ]:
#find players with quest
qTitle = raw_input("What Quest are you searching for: ")
cur.execute("SELECT gamerID, class, level FROM gamerQuests WHERE title=%s", [qTitle])
qTitles = cur.fetchall()


qt = pd.DataFrame( [[ij for ij in i] for i in qTitles] )
qt.rename(columns={0: 'Gamer Name', 1: 'Class', 2: 'Level'}, inplace=True)
qt.head(100)